## Homework

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

--2021-04-11 15:24:56--  https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX
Resolving docs.google.com (docs.google.com)... 172.217.164.174, 2607:f8b0:4004:832::200e
Connecting to docs.google.com (docs.google.com)|172.217.164.174|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/krf2r90tejulnmknvkhdpt7u17pabitr/1618154625000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download [following]
--2021-04-11 15:24:58--  https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/krf2r90tejulnmknvkhdpt7u17pabitr/1618154625000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download
Resolving doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleusercontent.com)... 172.217.15.65, 2607:f8b0:4004:810::2001
Connecting to doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.goog

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])

In [ ]:
len(df)*0.8

16000.0

Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки по следующему принципу:
```
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
```

In [ ]:
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))

In [ ]:
num_df = df[num_columns]

In [ ]:
num_df.isna().sum()

hospital_beds_raion            9404
leisure_count_2000                0
additional_education_km           0
bulvar_ring_km                    0
cafe_count_1500_price_4000        0
                               ... 
build_count_wood               3175
children_preschool                0
cafe_sum_1000_max_price_avg    4285
hospice_morgue_km                 0
office_raion                      0
Length: 266, dtype: int64

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np

X_train, y_train = num_df.drop('price', axis=1)[:16000], df['price'][:16000]
X_test, y_test = num_df.drop('price', axis=1)[16000: ], df['price'][16000: ]

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X_train)
X_train = pd.DataFrame(data=imputer.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(data=imputer.transform(X_test), columns=X_test.columns)

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(random_state=42)
model.fit(X_train.values, y_train.values)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

Проверьте качество на отложенной выборке.

In [ ]:
from sklearn.metrics import mean_squared_error

preds = model.predict(X_test.values)

print(f'RMSE: {mean_squared_error(y_test, preds)**0.5}')

RMSE: 3770456.3009582446


### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [ ]:
month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df["week_year_cnt"] = week_year.map(week_year_cnt_map)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [ ]:
month = df.timestamp.dt.month
day = df.timestamp.dt.weekday
fl_max_f = df.floor / df.max_floor
ksq_fsq = df.kitch_sq / df.full_sq

In [ ]:
df.month = month
df.day = day
df.fl_max_f = fl_max_f
df.ksq_fsq = ksq_fsq

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-

Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [ ]:
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))

X_train, y_train = num_df.drop('price', axis=1)[:16000], df['price'][:16000]
X_test, y_test = num_df.drop('price', axis=1)[16000: ], df['price'][16000: ]

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X_train)
X_train = pd.DataFrame(data=imputer.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(data=imputer.transform(X_test), columns=X_test.columns)

### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

In [ ]:
from sklearn.model_selection import GridSearchCV

parametrs = {'max_depth': [2, 3, 5, 10], 'min_samples_split': [1, 2, 5], 
             'min_samples_leaf': [1, 2, 5] }
model = DecisionTreeRegressor(random_state=42)
gs_dt = GridSearchCV(model, parametrs, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
gs_dt.fit(X_train, y_train)



Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:   38.3s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=42, splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': [2, 3, 5, 10],
                         'min_samples_leaf': [1, 2, 5],
                         'min_samples_split': [1, 2, 5]},
             pre_dispatch='2*n_jobs',

In [ ]:
preds = gs_dt.predict(X_test)
print(f'RMSE: {mean_squared_error(y_test, preds)**0.5}')

RMSE: 3124314.6014944855


In [ ]:
from sklearn.ensemble import RandomForestRegressor

param_grid = {'n_estimators': [50, 100, 200],
              'min_samples_split':[1, 2, 5],
              'min_samples_leaf': [1, 2, 5],
              'max_depth': [5, 10]}

model = RandomForestRegressor()
# Instantiate the grid search model
gs_rf = GridSearchCV(model, param_grid, 
                          cv = 3,scoring='neg_mean_squared_error', n_jobs = -1, verbose = 1)

gs_rf.fit(X_train, y_train)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


KeyboardInterrupt: ignored

In [ ]:
!pip install catboost

     |████████████████████████████████| 67.3MB 62kB/s 


In [ ]:
from catboost import CatBoostRegressor

In [ ]:

model = CatBoostRegressor()

grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

grid_search_result = model.grid_search(grid, 
                                       X=X_train, 
                                       y=y_train, 
                                       plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Выходные данные были обрезаны до нескольких последних строк (5000).
21:	learn: 5326397.7510054	test: 5252946.1518742	best: 5252946.1518742 (21)	total: 10.6s	remaining: 7m 49s
22:	learn: 5231557.5769321	test: 5159216.7534710	best: 5159216.7534710 (22)	total: 11.1s	remaining: 7m 49s
23:	learn: 5144815.3003694	test: 5074972.9043071	best: 5074972.9043071 (23)	total: 11.5s	remaining: 7m 48s
24:	learn: 5049856.3284536	test: 4979934.5580508	best: 4979934.5580508 (24)	total: 12s	remaining: 7m 48s
25:	learn: 4959663.8480742	test: 4889858.4578016	best: 4889858.4578016 (25)	total: 12.5s	remaining: 7m 48s
26:	learn: 4876949.3193260	test: 4808148.8602225	best: 4808148.8602225 (26)	total: 13s	remaining: 7m 47s
27:	learn: 4789516.6801660	test: 4720930.9421952	best: 4720930.9421952 (27)	total: 13.4s	remaining: 7m 46s
28:	learn: 4712940.5223534	test: 4647817.1250978	best: 4647817.1250978 (28)	total: 13.9s	remaining: 7m 46s
29:	learn: 4635373.9169912	test: 4573230.8335815	best: 4573230.8335815 (29)	tota

In [ ]:
grid_search_result['params']

{'depth': 10, 'l2_leaf_reg': 5, 'learning_rate': 0.03}

In [ ]:
model = CatBoostRegressor(depth = 10, l2_leaf_reg = 5, learning_rate = 0.03)

model.fit(X_train, y_train)

preds = model.predict(X_test)
print(f'RMSE: {mean_squared_error(y_test, preds)**0.5}')

0:	learn: 4529272.0366858	total: 479ms	remaining: 7m 58s
1:	learn: 4459928.3544411	total: 1.02s	remaining: 8m 31s
2:	learn: 4397055.3953505	total: 1.51s	remaining: 8m 23s
3:	learn: 4331424.0121872	total: 2.01s	remaining: 8m 20s
4:	learn: 4272390.5183068	total: 2.5s	remaining: 8m 16s
5:	learn: 4213853.2358467	total: 2.98s	remaining: 8m 14s
6:	learn: 4156911.0254908	total: 3.47s	remaining: 8m 12s
7:	learn: 4102560.0136545	total: 4.02s	remaining: 8m 18s
8:	learn: 4056117.9303631	total: 4.55s	remaining: 8m 21s
9:	learn: 4007567.0233872	total: 5.05s	remaining: 8m 20s
10:	learn: 3962130.9245268	total: 5.54s	remaining: 8m 18s
11:	learn: 3913405.4634003	total: 6.04s	remaining: 8m 17s
12:	learn: 3865190.1616563	total: 6.53s	remaining: 8m 15s
13:	learn: 3819203.6949661	total: 7.06s	remaining: 8m 17s
14:	learn: 3777019.4046340	total: 7.54s	remaining: 8m 15s
15:	learn: 3738298.1719152	total: 8.04s	remaining: 8m 14s
16:	learn: 3697989.0412925	total: 8.53s	remaining: 8m 13s
17:	learn: 3660359.969047

### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

In [ ]:
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))

invest_train_X = df.loc[np.where(df.product_type == 'Investment')[0], num_columns].drop('price', axis = 1)
invest_train_Y = df.loc[(np.where(df.product_type == 'Investment')[0]), 'price']

owner_train_X = df.loc[(np.where(df.product_type == 'OwnerOccupier')[0]), num_columns].drop('price', axis = 1)
owner_train_Y = df.loc[(np.where(df.product_type == 'OwnerOccupier')[0]), 'price']

model_invest = CatBoostRegressor(depth = 10, l2_leaf_reg = 5, learning_rate = 0.03)
model_invest.fit(invest_train_X.values, invest_train_Y.values)

model_owner = CatBoostRegressor(depth = 10, l2_leaf_reg = 5, learning_rate = 0.03)
model_owner.fit(owner_train_X.values, owner_train_Y.values)

X_test, y_test = df[num_columns].drop('price', axis = 1)[16000: ], df['price'][16000: ]
preds1 = model_invest.predict(X_test)
preds2 = model_owner.predict(X_test)
preds = (preds1 + preds2) / 2

print(f'RMSE: {mean_squared_error(y_test, preds)**0.5}')

0:	learn: 4824117.6410331	total: 527ms	remaining: 8m 46s
1:	learn: 4758802.9413251	total: 1.03s	remaining: 8m 33s
2:	learn: 4695265.6133646	total: 1.56s	remaining: 8m 37s
3:	learn: 4630450.5229984	total: 2.06s	remaining: 8m 31s
4:	learn: 4575520.5932916	total: 2.55s	remaining: 8m 28s
5:	learn: 4519513.5513447	total: 3.06s	remaining: 8m 26s
6:	learn: 4465100.6959231	total: 3.58s	remaining: 8m 27s
7:	learn: 4408936.0541844	total: 4.09s	remaining: 8m 27s
8:	learn: 4360611.0287096	total: 4.59s	remaining: 8m 25s
9:	learn: 4310358.5004763	total: 5.1s	remaining: 8m 24s
10:	learn: 4268372.0428993	total: 5.6s	remaining: 8m 23s
11:	learn: 4222198.5481382	total: 6.11s	remaining: 8m 22s
12:	learn: 4176520.9181260	total: 6.64s	remaining: 8m 24s
13:	learn: 4130615.8257963	total: 7.13s	remaining: 8m 22s
14:	learn: 4085348.2804869	total: 7.63s	remaining: 8m 21s
15:	learn: 4040751.4104686	total: 8.15s	remaining: 8m 21s
16:	learn: 4003392.0455896	total: 8.66s	remaining: 8m 20s
17:	learn: 3968637.6792519

### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).

In [ ]:
from sklearn.linear_model import HuberRegressor

num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

invest_train_X = df.loc[np.where(df.product_type == 'Investment')[0], num_columns].drop('price', axis = 1)
invest_train_Y = df.loc[(np.where(df.product_type == 'Investment')[0]), 'price']
imputer.fit(invest_train_X)
invest_train_X = pd.DataFrame(data=imputer.transform(invest_train_X), columns=invest_train_X.columns)

owner_train_X = df.loc[(np.where(df.product_type == 'OwnerOccupier')[0]), num_columns].drop('price', axis = 1)
owner_train_Y = df.loc[(np.where(df.product_type == 'OwnerOccupier')[0]), 'price']
imputer.fit(owner_train_X)
owner_train_X = pd.DataFrame(data=imputer.transform(owner_train_X), columns=owner_train_X.columns)

model_invest1 = CatBoostRegressor(depth = 10, l2_leaf_reg = 5, learning_rate = 0.03)
model_invest2 = HuberRegressor()
model_invest1.fit(invest_train_X.values, invest_train_Y.values)
model_invest2.fit(invest_train_X.values, invest_train_Y.values)

model_owner = CatBoostRegressor(depth = 10, l2_leaf_reg = 5, learning_rate = 0.03)
model_owner.fit(owner_train_X.values, owner_train_Y.values)

X_test, y_test = df[num_columns].drop('price', axis = 1)[16000: ], df['price'][16000: ]
imputer.fit(X_test)
X_test = pd.DataFrame(data=imputer.transform(X_test), columns=X_test.columns)

preds11 = model_invest1.predict(X_test)
preds12 = model_invest2.predict(X_test)
preds2 = model_owner.predict(X_test)

preds1 = 0.8*preds11 + 0.2*preds12
preds = (preds1 + preds2) / 2

print(f'RMSE: {mean_squared_error(y_test, preds)**0.5}')

RMSE: 2485273.5128194606
